In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
data = pd.read_csv('postal_data.csv')
print(data.head())

  Postcode      Borough                         Neighbourhood   Latitude  \
0      M1B  Scarborough                         Rouge,Malvern  43.806686   
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union  43.784535   
2      M1E  Scarborough       Guildwood,Morningside,West Hill  43.763573   
3      M1G  Scarborough                                Woburn  43.770992   
4      M1H  Scarborough                             Cedarbrae  43.773136   

   Longitude  
0 -79.194353  
1 -79.160497  
2 -79.188711  
3 -79.216917  
4 -79.239476  


In [3]:
address = 'Toronto , ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [4]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [5]:
CLIENT_ID = 'WO5VHHRQ1YQCISVZ1RHUPMPMKQV1EL3HLUCFJE02N4YIX01E' # your Foursquare ID
CLIENT_SECRET = 'OZGOUJU3PA2BRQA4HNNXCAWBJTCP33WS2SXXER552SXVWF5A' # your Foursquare Secret
VERSION = '20190901' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WO5VHHRQ1YQCISVZ1RHUPMPMKQV1EL3HLUCFJE02N4YIX01E
CLIENT_SECRET:OZGOUJU3PA2BRQA4HNNXCAWBJTCP33WS2SXXER552SXVWF5A


In [6]:
data.loc[0, 'Neighbourhood']

'Rouge,Malvern'

In [7]:
neighborhood_latitude = data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = data.loc[0, 'Neighbourhood'] # neighborhood name


In [8]:
LIMIT = 100
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=WO5VHHRQ1YQCISVZ1RHUPMPMKQV1EL3HLUCFJE02N4YIX01E&client_secret=OZGOUJU3PA2BRQA4HNNXCAWBJTCP33WS2SXXER552SXVWF5A&v=20190901&ll=43.8066863,-79.19435340000003&radius=500&limit=100'

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d98eddea879210039a484ee'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 43.811186304500005,
    'lng': -79.18812958073043},
   'sw': {'lat': 43.8021862955, 'lng': -79.20057721926962}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': "Wendy's",
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'di

In [10]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [11]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

name            categories        lat        lng
0  Wendy's  Fast Food Restaurant  43.807448 -79.199056

### create function to call 4square API to avoid recoding

In [12]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### run function

In [13]:
toronto_venues = getNearbyVenues(names=data['Neighbourhood'],
                                   latitudes=data['Latitude'],
                                   longitudes=data['Longitude']
                                  )

Rouge,Malvern
Highland Creek,Rouge Hill,Port Union
Guildwood,Morningside,West Hill
Woburn
Cedarbrae
Scarborough Village
East Birchmount Park,Ionview,Kennedy Park
Clairlea,Golden Mile,Oakridge
Cliffcrest,Cliffside,Scarborough Village West
Birch Cliff,Cliffside West
Dorset Park,Scarborough Town Centre,Wexford Heights
Maryvale,Wexford
Agincourt
Clarks Corners,Sullivan,Tam O'Shanter
Agincourt North,L'Amoreaux East,Milliken,Steeles East
L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview,Henry Farm,Oriole
Bayview Village
Silver Hills,York Mills
Newtonbrook,Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park,Don Mills South
Bathurst Manor,Downsview North,Wilson Heights
Northwood Park,York University
CFB Toronto,Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Woodbine Gardens,Parkview Hill
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto
The Danforth West,Riverdale
The Beaches West,Indi

In [14]:
print(toronto_venues.shape)
toronto_venues.head()

(2246, 7)


Neighborhood  Neighborhood Latitude  \
0                         Rouge,Malvern              43.806686   
1  Highland Creek,Rouge Hill,Port Union              43.784535   
2       Guildwood,Morningside,West Hill              43.763573   
3       Guildwood,Morningside,West Hill              43.763573   
4       Guildwood,Morningside,West Hill              43.763573   

   Neighborhood Longitude                            Venue  Venue Latitude  \
0              -79.194353                          Wendy's       43.807448   
1              -79.160497            Royal Canadian Legion       43.782533   
2              -79.188711  Swiss Chalet Rotisserie & Grill       43.767697   
3              -79.188711                G & G Electronics       43.765309   
4              -79.188711                 Big Bite Burrito       43.766299   

   Venue Longitude        Venue Category  
0       -79.199056  Fast Food Restaurant  
1       -79.163085                   Bar  
2       -79.189914           Pizza Place  
3       -79.191537     Electronics Store  
4       -79.190720    Mexican Restaurant

In [15]:
print('There are {} categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 categories.


In [16]:
# process data to normalize value
toronto_oh = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neigbor to datafram
toronto_oh['Neighborhood'] = toronto_venues['Neighborhood'] 

# change column arrangement
fixed_columns = [toronto_oh.columns[-1]] + list(toronto_oh.columns[:-1])
toronto_oh = toronto_oh[fixed_columns]

toronto_oh.head()

Yoga Studio  Accessories Store  Afghan Restaurant  Airport  \
0            0                  0                  0        0   
1            0                  0                  0        0   
2            0                  0                  0        0   
3            0                  0                  0        0   
4            0                  0                  0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  American Restaurant  Antique Shop  Aquarium  Art Gallery  \
0                 0                    0             0         0            0   
1                 0                    0             0         0            0   
2                 0                    0             0         0            0   
3                 0                    0             0         0            0   
4                 0                    0             0         0            0   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Auto Garage  \
0                    0                 0                   0            0   
1                    0                 0                   0            0   
2                    0                 0                   0            0   
3                    0                 0                   0            0   
4                    0                 0                   0            0   

   Auto Workshop  BBQ Joint  Baby Store  Bagel Shop  Bakery  Bank  Bar  \
0              0          0           0           0       0     0    0   
1              0          0           0           0       0     0    1   
2              0          0           0           0       0     0    0   
3              0          0           0           0       0     0    0   
4              0          0           0           0       0     0    0   

   Baseball Field  Baseball Stadium  Basketball Court  Basketball Stadium  \
0               0                 0                 0                   0   
1               0                 0                 0                   0   
2               0                 0                 0                   0   
3               0                 0                 0                   0   
4               0                 0                 0                   0   

   Beach  Bed & Breakfast  Beer Bar  Beer Store  Belgian Restaurant  \
0      0                0         0           0                   0   
1      0                0         0           0                   0   
2      0                0         0           0                   0   
3      0                0         0           0                   0   
4      0                0         0           0                   0   

   Bike Shop  Bistro  Boat or Ferry  Bookstore  Boutique  Bowling Alley  \
0          0       0              0          0         0              0   
1          0       0              0          0         0              0   
2          0       0              0          0         0              0   
3          0       0              0          0         0              0   
4          0       0              0          0         0              0   

   Brazilian Restaurant  Breakfast Spot  Brewery  Bridal Shop  \
0                     0               0        0            0   
1                     0               0        0            0   
2                     0               0        0            0   
3                     0               0        0            0   
4                     0               0        0            0   

   Bubble Tea Shop  Building  Burger Joint  Burrito 

In [18]:
#group by neighborhood
toronto_g = toronto_oh.groupby('Neighborhood').mean().reset_index()
toronto_g

Neighborhood  Yoga Studio  \
0                              Adelaide,King,Richmond     0.000000   
1                                           Agincourt     0.000000   
2   Agincourt North,L'Amoreaux East,Milliken,Steel...     0.000000   
3   Albion Gardens,Beaumond Heights,Humbergate,Jam...     0.000000   
4                               Alderwood,Long Branch     0.000000   
5       Bathurst Manor,Downsview North,Wilson Heights     0.000000   
6                                     Bayview Village     0.000000   
7                    Bedford Park,Lawrence Manor East     0.000000   
8                                         Berczy Park     0.000000   
9                          Birch Cliff,Cliffside West     0.000000   
10  Bloordale Gardens,Eringate,Markland Wood,Old B...     0.000000   
11         Brockton,Exhibition Place,Parkdale Village     0.000000   
12  Business Reply Mail Processing Centre 969 Eastern     0.066667   
13                         CFB Toronto,Downsview East     0.000000   
14  CN Tower,Bathurst Quay,Island airport,Harbourf...     0.000000   
15                         Cabbagetown,St. James Town     0.000000   
16                                Caledonia-Fairbanks     0.000000   
17              Canada Post Gateway Processing Centre     0.000000   
18                                          Cedarbrae     0.000000   
19                                 Central Bay Street     0.011765   
20            Chinatown,Grange Park,Kensington Market     0.000000   
21                                           Christie     0.000000   
22                               Church and Wellesley     0.011628   
23                      Clairlea,Golden Mile,Oakridge     0.000000   
24              Clarks Corners,Sullivan,Tam O'Shanter     0.000000   
25      Cliffcrest,Cliffside,Scarborough Village West     0.000000   
26                      Commerce Court,Victoria Hotel     0.000000   
27                                         Davisville     0.000000   
28                                   Davisville North     0.000000   
29  Deer Park,Forest Hill SE,Rathnelly,South Hill,...     0.000000   
30        Del Ray,Keelesdale,Mount Dennis,Silverthorn     0.000000   
31            Design Exchange,Toronto Dominion Centre     0.000000   
32                                    Don Mills North     0.000000   
33  Dorset Park,Scarborough Town Centre,Wexford He...     0.000000   
34                        Dovercourt Village,Dufferin     0.000000   
35                                  Downsview Central     0.000000   
36                                Downsview Northwest     0.000000   
37                                     Downsview West     0.000000   
38                   Downsview,North Park,Upwood Park     0.000000   
39          East Birchmount Park,Ionview,Kennedy Park     0.000000   
40                                       East Toronto     0.000000   
41                                    Emery,Humberlea     0.000000   
42                         Fairview,Henry Farm,Oriole     0.000000   
43              First Canadian Place,Underground city     0.000000   
44                    Flemingdon Park,Don Mills South     0.000000   
45                 Forest Hill North,Forest Hill West     0.000000   
46                                          Glencairn     0.000000   
47                    Guildwood,Morningside,West Hill     0.000000   
48                      Harbord,University of Toronto     0.000000   
49    Harbourfront East,Toronto Islands,Union Station     0.000000   
50                           Harbourfront,Regent Park     0.020000   
51                       High Park,The Junction South     0.000000   
52               Highland Creek,Rouge Hill,Port Union     0.000000   
53                                  Hillcrest Village     0.000000   
54         Humber Bay Shores,Mimico South,New Toronto     0.000000   
55  Humber Bay,King's Mill Park,Kingsway Park Sout...     0.000000   
56                                      Humber Summ

### print top 3 venues

In [22]:
top_venues = 3

for h in toronto_g['Neighborhood']:
    print("----"+h+"----")
    temp = toronto_g[toronto_g['Neighborhood'] == h].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(top_venues))
    print('\n')

----Adelaide,King,Richmond----
                 venue  freq
0          Coffee Shop  0.08
1                 Café  0.05
2  American Restaurant  0.04


----Agincourt----
            venue  freq
0    Skating Rink  0.25
1          Lounge  0.25
2  Breakfast Spot  0.25


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                venue  freq
0          Playground   0.5
1                Park   0.5
2  Mexican Restaurant   0.0


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
           venue  freq
0  Grocery Store  0.22
1    Pizza Place  0.11
2   Liquor Store  0.11


----Alderwood,Long Branch----
            venue  freq
0     Pizza Place  0.18
1        Pharmacy  0.09
2  Sandwich Place  0.09


----Bathurst Manor,Downsview North,Wilson Heights----
                  venue  freq
0           Coffee Shop  0.10
1             Pet Store  0.05
2  Fast Food Restaurant  0.05


----Bayview Village----
                 venue  fr

              venue  freq
0  Greek Restaurant  0.19
1       Coffee Shop  0.09
2    Ice Cream Shop  0.07


----The Junction North,Runnymede----
                  venue  freq
0  Caribbean Restaurant  0.33
1               Brewery  0.33
2              Bus Line  0.33


----The Kingsway,Montgomery Road,Old Mill North----
   venue  freq
0  River  0.25
1   Park  0.25
2   Pool  0.25


----Thorncliffe Park----
               venue  freq
0  Indian Restaurant  0.12
1       Burger Joint  0.12
2        Yoga Studio  0.06


----Victoria Village----
               venue  freq
0       Intersection  0.17
1        Pizza Place  0.17
2  French Restaurant  0.17


----Westmount----
            venue  freq
0     Pizza Place  0.29
1  Discount Store  0.14
2  Sandwich Place  0.14


----Weston----
                venue  freq
0                Park   1.0
1         Yoga Studio   0.0
2  Mexican Restaurant   0.0


----Willowdale South----
              venue  freq
0  Ramen Restaurant  0.09
1       Coffee Shop  0.09
2  

### create function for top venue

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### get top 5 catgory venue for all

In [107]:
top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_g['Neighborhood']

for ind in np.arange(toronto_g.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_g.iloc[ind, :], top_venues)

neighborhoods_venues_sorted.head()

Neighborhood 1st Most Common Venue  \
0                             Adelaide,King,Richmond           Coffee Shop   
1                                          Agincourt                Lounge   
2  Agincourt North,L'Amoreaux East,Milliken,Steel...            Playground   
3  Albion Gardens,Beaumond Heights,Humbergate,Jam...         Grocery Store   
4                              Alderwood,Long Branch           Pizza Place   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
0                  Café   American Restaurant                   Bar   
1        Clothing Store          Skating Rink        Breakfast Spot   
2                  Park         Women's Store            Donut Shop   
3          Liquor Store              Pharmacy           Pizza Place   
4              Pharmacy        Sandwich Place                   Gym   

  5th Most Common Venue  
0            Steakhouse  
1            Donut Shop  
2          Dessert Shop  
3   Fried Chicken Joint  
4           Coffee Shop

### start clustering

In [193]:
# set number of clusters
kclusters = 5

toronto_g_clustering = toronto_g.drop('Neighborhood', 1)

run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2).fit(toronto_g_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

Neighborhood  Yoga Studio  Accessories Store  Afghan Restaurant  \
36  Downsview Northwest          0.0                0.0                0.0   
69            Northwest          0.0                0.0                0.0   

    Airport  Airport Food Court  Airport Gate  Airport Lounge  \
36      0.0                 0.0           0.0             0.0   
69      0.0                 0.0           0.0             0.0   

    Airport Service  Airport Terminal  American Restaurant  Antique Shop  \
36              0.0               0.0                  0.0           0.0   
69              0.0               0.0                  0.0           0.0   

    Aquarium  Art Gallery  Arts & Crafts Store  Asian Restaurant  \
36       0.0          0.0                  0.0               0.0   
69       0.0          0.0                  0.0               0.0   

    Athletics & Sports  Auto Garage  Auto Workshop  BBQ Joint  Baby Store  \
36                 0.0          0.0            0.0        0.0         0.0   
69                 0.0          0.0            0.0        0.0         0.0   

    Bagel Shop  Bakery  Bank       Bar  Baseball Field  Baseball Stadium  \
36         0.0     0.0   0.0  0.000000             0.0               0.0   
69         0.0     0.0   0.0  0.333333             0.0               0.0   

    Basketball Court  Basketball Stadium  Beach  Bed & Breakfast  Beer Bar  \
36               0.0                 0.0    0.0              0.0       0.0   
69               0.0                 0.0    0.0              0.0       0.0   

    Beer Store  Belgian Restaurant  Bike Shop  Bistro  Boat or Ferry  \
36         0.0                 0.0        0.0     0.0            0.0   
69         0.0                 0.0        0.0     0.0            0.0   

    Bookstore  Boutique  Bowling Alley  Brazilian Restaurant  Breakfast Spot  \
36        0.0       0.0            0.0                   0.0             0.0   
69        0.0       0.0            0.0                   0.0             0.0   

    Brewery  Bridal Shop  Bubble Tea Shop  Building  Burger Joint  \
36      0.0          0.0              0.0       0.0           0.0   
69      0.0          0.0              0.0       0.0           0.0   

    Burrito Place  Bus Line  Bus Station  Bus Stop  Business Service  Butcher  \
36            0.0       0.0          0.0       0.0               0.0      0.0   
69            0.0       0.0          0.0       0.0               0.0      0.0   

    Café  Cajun / Creole Restaurant  Camera Store  Candy Store  \
36   0.0                        0.0           0.0          0.0   
69   0.0                        0.0           0.0          0.0   

    Caribbean Restaurant  Cheese Shop  Chinese Restaurant  Chocolate Shop  \
36                   0.0          0.0                 0.0             0.0   
69                   0.0          0.0                 0.0             0.0   

    Church  Climbing Gym  Clothing Store  Cocktail Bar  Coffee Shop  \
36     0.0           0.0             0.0           0.0          0.0   
69     0.0           0.0             0.0           0.0          0.0   

    College Arts Building  College Auditorium  College Gym  \
36                    0.0                 0.0          0.0   
69                    0.0                 0.0          0.0   

    College Rec Center  College Stadium  Colombian Restaurant  \
36                 0.0              0.0                   0.0   
69                 0.0              0.0                   0.0   

    Comfort Food Restaurant  Comic Shop  Concert Hall  \
36                      0.0         0.0           0.0   
69                      0.0         0.0           0.0   

    Construction & Landscaping  Convenience Store  Cosmetics Shop  \
36                         0.0                0.0             0.0   
69                         0.0                0.0             0.0   

    Coworking Space  Creperie  Cuban Restaurant  Cupcake Shop  Curling Ice  \
36              0.0       0.0               0.0       

In [161]:
#neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

### replace nan to 0
toronto_merged = toronto_merged.fillna(0)

### change float types to integer
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)

toronto_merged

Postcode           Borough  \
0        M1B       Scarborough   
1        M1C       Scarborough   
2        M1E       Scarborough   
3        M1G       Scarborough   
4        M1H       Scarborough   
5        M1J       Scarborough   
6        M1K       Scarborough   
7        M1L       Scarborough   
8        M1M       Scarborough   
9        M1N       Scarborough   
10       M1P       Scarborough   
11       M1R       Scarborough   
12       M1S       Scarborough   
13       M1T       Scarborough   
14       M1V       Scarborough   
15       M1W       Scarborough   
16       M1X       Scarborough   
17       M2H        North York   
18       M2J        North York   
19       M2K        North York   
20       M2L        North York   
21       M2M        North York   
22       M2N        North York   
23       M2P        North York   
24       M2R        North York   
25       M3A        North York   
26       M3B        North York   
27       M3C        North York   
28       M3H        North York   
29       M3J        North York   
30       M3K        North York   
31       M3L        North York   
32       M3M        North York   
33       M3N        North York   
34       M4A        North York   
35       M4B         East York   
36       M4C         East York   
37       M4E      East Toronto   
38       M4G         East York   
39       M4H         East York   
40       M4J         East York   
41       M4K      East Toronto   
42       M4L      East Toronto   
43       M4M      East Toronto   
44       M4N   Central Toronto   
45       M4P   Central Toronto   
46       M4R   Central Toronto   
47       M4S   Central Toronto   
48       M4T   Central Toronto   
49       M4V   Central Toronto   
50       M4W  Downtown Toronto   
51       M4X  Downtown Toronto   
52       M4Y  Downtown Toronto   
53       M5A  Downtown Toronto   
54       M5B  Downtown Toronto   
55       M5C  Downtown Toronto   
56       M5E  Downtown Toronto   
57       M5G  Downtown Toronto   
58       M5H  Downtown Toronto   
59       M5J  Downtown Toronto   
60       M5K  Downtown Toronto   
61       M5L  Downtown Toronto   
62       M5M        North York   
63       M5N   Central Toronto   
64       M5P   Central Toronto   
65       M5R   Central Toronto   
66       M5S  Downtown Toronto   
67       M5T  Downtown Toronto   
68       M5V  Downtown Toronto   
69       M5W  Downtown Toronto   
70       M5X  Downtown Toronto   
71       M6A        North York   
72       M6B        North York   
73       M6C              York   
74       M6E              York   
75       M6G  Downtown Toronto   
76       M6H      West Toronto   
77       M6J      West Toronto   
78       M6K      West Toronto   
79       M6L        North York   
80       M6M              York   
81       M6N              York   
82       M6P      West Toronto   
83       M6R      West Toronto   
84       M6S      West Toronto   
85       M7A      Queen's Park   
86       M7R       Mississauga   
87       M7Y      East Toronto   
88       M8V         Etobicoke   
89       M8W         Etobicoke   
90       M8X         Etobicoke   
91       M8Y         Etobicoke   
92       M8Z         Etobicoke   
93       M9A         Etobicoke   
94       M9B         Etobicoke   
95       M9C         Etobicoke   
96       M9L        North York   
97       M9M        North York   
98       M9N              York   
99       M9P         Etobicoke   
100      M9R         Etobicoke   
101      M9V         Etobicoke   
102      M9W         Etobicoke   

                                         Neighbourhood   Latitude  Longitude  \
0                                        Rouge,Malvern  43.806686 -79.194353   
1                 Highland Creek,Rouge Hill,Port Union  43.784535 -79.160497   
2                      Guildwood,Morningside,West Hill  43.763573 -79.188711   
3                                               Woburn  43.770992 -79.216917   
4                                            Cedarbrae  43.773136 -

In [162]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster*1],
        fill=True,
        fill_color=rainbow[cluster*1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 1st clusters consist of locations with multiple Venues

In [163]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels      1st Most Common Venue  \
1         Scarborough               0                        Bar   
2         Scarborough               0             Medical Center   
3         Scarborough               0                Coffee Shop   
4         Scarborough               0           Hakka Restaurant   
5         Scarborough               0                 Playground   
6         Scarborough               0                Coffee Shop   
7         Scarborough               0                   Bus Line   
8         Scarborough               0                      Motel   
9         Scarborough               0            College Stadium   
10        Scarborough               0          Indian Restaurant   
11        Scarborough               0  Middle Eastern Restaurant   
12        Scarborough               0                     Lounge   
13        Scarborough               0                Pizza Place   
15        Scarborough               0       Fast Food Restaurant   
16        Scarborough               0                          0   
17         North York               0   Mediterranean Restaurant   
18         North York               0             Clothing Store   
19         North York               0                       Café   
20         North York               0                          0   
22         North York               0           Ramen Restaurant   
24         North York               0              Grocery Store   
26         North York               0       Gym / Fitness Center   
27         North York               0                Coffee Shop   
28         North York               0                Coffee Shop   
29         North York               0         Falafel Restaurant   
31         North York               0              Grocery Store   
32         North York               0           Business Service   
33         North York               0               Liquor Store   
34         North York               0               Hockey Arena   
35          East York               0       Fast Food Restaurant   
36          East York               0               Skating Rink   
37       East Toronto               0                      Trail   
38          East York               0                Coffee Shop   
39          East York               0          Indian Restaurant   
41       East Toronto               0           Greek Restaurant   
42       East Toronto               0             Sandwich Place   
43       East Toronto               0                       Café   
44    Central Toronto               0         Photography Studio   
45    Central Toronto               0                        Gym   
46    Central Toronto               0                Coffee Shop   
47    Central Toronto               0               Dessert Shop   
49    Central Toronto               0                Coffee Shop   
51   Downtown Toronto               0                Coffee Shop   
52   Downtown Toronto               0                Coffee Shop   
53   Downtown Toronto               0                Coffee Shop   
54   Downtown Toronto               0                Coffee Shop   
55   Downtown Toronto               0                       Café   
56   Downtown Toronto               0                Coffee Shop   
57   Downtown Toronto               0                Coffee Shop   
58   Downtown Toronto               0                Coffee Shop   
59   Downtown Toronto               0                Coffee Shop   
60   Downtown Toronto               0                Coffee Shop   
61   Downtown Toronto               0                Coffee Shop   
62         North York               0                Coffee Shop   
63    Central Toronto               0                     Garden   
64    Central Toronto               0                       Park   
65    Central Toronto               0             Sandwich Place   
66   Downtown Toronto               0                       Café   
67

In [164]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
0  Scarborough               1  Fast Food Restaurant      Department Store   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  
0           Event Space  Ethiopian Restaurant   Empanada Restaurant

In [165]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue 2nd Most Common Venue  \
21  North York               2                   Gym      Department Store   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  
21           Event Space  Ethiopian Restaurant   Empanada Restaurant

In [166]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels   1st Most Common Venue 2nd Most Common Venue  \
91   Etobicoke               3          Baseball Field         Women's Store   
97  North York               3  Furniture / Home Store        Baseball Field   

   3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  
91    Dim Sum Restaurant                 Diner        Discount Store  
97         Women's Store      Department Store    Dim Sum Restaurant

### cluster with parks and women's stores

In [167]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Borough  Cluster Labels 1st Most Common Venue  \
14        Scarborough               4            Playground   
23         North York               4                  Park   
25         North York               4                  Park   
30         North York               4           Snack Place   
40          East York               4                  Park   
48    Central Toronto               4            Playground   
50   Downtown Toronto               4                  Park   
74               York               4                  Park   
98               York               4                  Park   
100         Etobicoke               4           Pizza Place   

    2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
14                   Park         Women's Store            Donut Shop   
23                   Bank     Convenience Store         Women's Store   
25      Food & Drink Shop         Women's Store            Donut Shop   
30                Airport                  Park         Women's Store   
40      Convenience Store           Coffee Shop   Dumpling Restaurant   
48                   Park            Restaurant         Women's Store   
50             Playground              Building                 Trail   
74          Women's Store  Fast Food Restaurant                Market   
98          Women's Store             Drugstore    Dim Sum Restaurant   
100     Mobile Phone Shop                  Park         Women's Store   

    5th Most Common Venue  
14           Dessert Shop  
23              Drugstore  
25     Dim Sum Restaurant  
30             Donut Shop  
40                  Diner  
48       Doner Restaurant  
50          Women's Store  
74             Donut Shop  
98                  Diner  
100      Doner Restaurant